# Designing Interfaces
Considerations for writing idiomatic interfaces in Rust (including how to name things, how to use the type system to enforce API contracts, and when to use generics versus trait objects)
## Unsurprising
The interface should be predictable according to the _Principle of Least Surprise_
### Naming practices
- things that share a name should in fact work the same way
- e.g. `iter`, `SomethingError`
### Common Traits for Types
- users expect common traits to be implemented
- user cannot implement missing traits (coherence rules)
- implement `Debug`, `Clone`, `Default` should be implemented
- avoid `Send`, `Sync` implementation only for good reasons
- implement `PartialEq`, `PartialOrd`, `Hash`, `Eq`, and `Ord` when possible
- `serde::Serialize` and `serde::Deserialize`: useful; define a `serde` feature to avoid forcing user to add the `serde` dependency
- pay attention to `Copy`: changes the move semantics (may surprise the user)
### Ergonomic Trait Implementations
Blanket implementations for reference types (not automatically provided by Rust) should be provided for:
- `&T where T: Trait`
- `&mut T where T: Trait`
- `Box<T> where T: Trait`
For iterators: implement `IntoIterator` for both `&MyType` and `&mut MyType`
### Wrapper Types
Consider implementing:
- `Deref`, for transparent wrapper types (like `Arc`), to allow access to the inner type with the `.` operator)
- `AsRef` to allow use a `&WrapperType` as an `&InnerType`
- `From<InnerType>` and `Into<InnerType>` to easily add or remove the wrapping.
- `Borrow` to allow supply an equivalent type (`Deref` and `AsRef` are intended to be implemented more widely for anything your type can “act as.”)
## Flexible
- contract: a set of requirements and a set of promises.
- avoid imposing unnecessary restrictions
### Generic Arguments
- use generics rather than concrete types
- start with the argument fully generic with no bounds, and then add the needed bounds (from compiler errors)
- for readabilty and performance, avoid generics when not actually needed
- consider dynamic dispatching when performance not an issue
- moving from concrete types to generics may break backward compatibility
### Object Safety
- implicitly part of the contract
- should be preserved, also at the cost of ergonomicity
- a future change will break backward compatibility
### Borrowed vs. Owned
- every defined function, trait, and type must choose whether it should own, or just hold a reference to data
- if the code needs ownership of the data, it must store the owned data (should be explicit on the interface)
- otherwise, it should operate on references (except for small types)
- if ownership requirements are unclear, use `Cow`
### Fallible and Blocking Destructors
- cleanup is usually done into `Drop`
- this don't allow to handle errors
- for fine control, expose an explicit destructor (must be documented)
## Obvious
### Documentation
- provide end-to-end examples
- use modules to group together semantically related items
- Use `#[doc(cfg(..))]` to highlight items that are available only under certain configurations
- use `#[doc(alias = "...")]` to make types and methods discoverable under other names
### Type System Guidance
- use _semantic typing_, where types represent the meaning of a value, rather than primitive type
- annotate with `#[must_use]` to prevent not using a returned value
- use `PanthomData` (erased at compile time) to prevent representing illegal states:

In [10]:
use std::marker::PhantomData;

struct Init;
struct Opened;

struct File<Status = Init> {
 stage: PhantomData<Status>,
}

impl Default for File<Init> {
    fn default() -> Self { File { stage: PhantomData } }
}

impl File<Init> {
    pub fn open(self) -> File<Opened> { 
        File { stage: PhantomData }
    }
}

impl File<Opened> {
    pub fn read(self) -> String { "this is not a file".to_owned() }
}

let file = File::default();

// won't compile
// print!("{}", file.read());

let opened_filed = file.open();
println!("{}", opened_filed.read());

this is not a file


## Constrained
- backward incompatible changes may be not obvious
### Type Modifications
- keep most types private to allow future changes
- adding a field to a struct will break constructors and pattern matching
- the `#[non_exhaustive]` force downstream crates to always match the `_` pattern
### Trait Implementations
- adding a new trait implementation may break compatibility, if contains an method also belonging to another trait
- _sealed traits_ can only be implemented within the crate that defines them. It is a pattern implemented by adding a private trait as a supertrait of the sealed trait

In [11]:
mod lib_mod {
    pub trait MyTrait: private::Sealed {}
    
    impl MyTrait for u8 {}
    
    mod private {
        pub trait Sealed {}
        impl Sealed for u8 {}
    }
}

mod client_mod {
    struct T;
    
    // won't compile
    // impl crate::lib_mod::MyTrait for T {}
}

### Hidden Contracts
- _re-exports_ make foreign crates part of the interface contract (and, thus, propagates the changes)
- moreover, in Rust, type with the same name from two major versions of the same crate are considered different types
- To mitigate the issue, wrap foreign types using the newtype pattern
- _auto-traits_ are part of the interface and are automatically added according to conditions that may change